<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [14]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [15]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        print(f"Rocket: {x}")
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [16]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         print(f"Launchpad {x}")
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [17]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        print(f"Payload: {load}")
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [18]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                print(f"Core: {core['core']}")
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [19]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [20]:
response = requests.get(spacex_url)

Check the content of the response


You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [21]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [22]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [23]:
# Use json_normalize meethod to convert the json result into a dataframe
response_json = response.json()
launch_df = pd.json_normalize(response_json)

Using the dataframe <code>data</code> print the first 5 rows


In [24]:
# Get the head of the dataframe
print(launch_df.head(10))
launch_df.describe()
data = launch_df

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   
5  2010-03-13T00:00:00.000Z           1.268438e+09  False     0.0   
6  2010-12-04T00:00:00.000Z           1.291421e+09  False     0.0   
7  2012-04-30T00:00:00.000Z           1.335744e+09  False     0.0   
8  2012-09-29T00:00:00.000Z           1.348877e+09  False     0.0   
9  2013-02-25T18:30:00.000Z           1.361817e+09  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   
5  5e9

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [25]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [26]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [27]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [28]:
# Call getBoosterVersion
getBoosterVersion(data)

Rocket: 5e9d0d95eda69955f709d1eb
Rocket: 5e9d0d95eda69955f709d1eb
Rocket: 5e9d0d95eda69955f709d1eb
Rocket: 5e9d0d95eda69955f709d1eb
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e9d0d95eda69973a809d1ec
Rocket: 5e

In [29]:
len(BoosterVersion)

94

the list has now been update 


In [30]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [31]:
# Call getLaunchSite
getLaunchSite(data)

Launchpad 5e9e4502f5090995de566f86
Launchpad 5e9e4502f5090995de566f86
Launchpad 5e9e4502f5090995de566f86
Launchpad 5e9e4502f5090995de566f86
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4502f509092b78566f87
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4502f509092b78566f87
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f509094ba4566f84
Launchpad 5e9e4501f5

In [32]:
# Call getPayloadData
getPayloadData(data)

Payload: 5eb0e4b5b6c3bb0006eeb1e1
Payload: 5eb0e4b6b6c3bb0006eeb1e2
Payload: 5eb0e4b7b6c3bb0006eeb1e5
Payload: 5eb0e4b7b6c3bb0006eeb1e6
Payload: 5eb0e4b7b6c3bb0006eeb1e7
Payload: 5eb0e4bab6c3bb0006eeb1ea
Payload: 5eb0e4bbb6c3bb0006eeb1ed
Payload: 5eb0e4bbb6c3bb0006eeb1ee
Payload: 5eb0e4bbb6c3bb0006eeb1ef
Payload: 5eb0e4bbb6c3bb0006eeb1f0
Payload: 5eb0e4bbb6c3bb0006eeb1f1
Payload: 5eb0e4bcb6c3bb0006eeb1f2
Payload: 5eb0e4bcb6c3bb0006eeb1f3
Payload: 5eb0e4bcb6c3bb0006eeb1f4
Payload: 5eb0e4bcb6c3bb0006eeb1f5
Payload: 5eb0e4bdb6c3bb0006eeb1f6
Payload: 5eb0e4bdb6c3bb0006eeb1f7
Payload: 5eb0e4bdb6c3bb0006eeb1fa
Payload: 5eb0e4beb6c3bb0006eeb1fb
Payload: 5eb0e4beb6c3bb0006eeb1fc
Payload: 5eb0e4beb6c3bb0006eeb1fd
Payload: 5eb0e4beb6c3bb0006eeb1fe
Payload: 5eb0e4beb6c3bb0006eeb1ff
Payload: 5eb0e4bfb6c3bb0006eeb200
Payload: 5eb0e4bfb6c3bb0006eeb201
Payload: 5eb0e4bfb6c3bb0006eeb202
Payload: 5eb0e4c0b6c3bb0006eeb205
Payload: 5eb0e4c1b6c3bb0006eeb206
Payload: 5eb0e4c1b6c3bb0006eeb207
Payload: 5eb0e

In [33]:
# Call getCoreData
getCoreData(data)

Core: 5e9e289df35918033d3b2623
Core: 5e9e289ef35918416a3b2624
Core: 5e9e289ef3591855dc3b2626
Core: 5e9e289ef359184f103b2627
Core: 5e9e289ef359185f2b3b2628
Core: 5e9e289ef35918f39c3b262a
Core: 5e9e289ff3591884e03b262c
Core: 5e9e289ff359180ae23b262d
Core: 5e9e289ff35918862c3b262e
Core: 5e9e289ff3591878603b262f
Core: 5e9e289ff3591829343b2630
Core: 5e9e28a0f3591870a63b2631
Core: 5e9e28a0f359186e2e3b2632
Core: 5e9e28a0f35918b1bc3b2633
Core: 5e9e28a0f359184a683b2634
Core: 5e9e28a0f359187a3c3b2635
Core: 5e9e28a0f3591885be3b2636
Core: 5e9e28a1f359186d533b2638
Core: 5e9e28a1f35918233f3b2639
Core: 5e9e28a1f35918683c3b263a
Core: 5e9e28a1f3591867753b263b
Core: 5e9e28a1f3591842fa3b263c
Core: 5e9e28a1f359188def3b263d
Core: 5e9e28a2f359182d0b3b263e
Core: 5e9e28a2f35918077b3b263f
Core: 5e9e28a2f3591845c73b2640
Core: 5e9e28a2f359187f273b2642
Core: 5e9e28a2f35918b8243b2643
Core: 5e9e28a2f359187ee83b2644
Core: 5e9e28a3f359189e3a3b2645
Core: 5e9e28a3f3591829dc3b2646
Core: 5e9e28a3f3591878473b2647
Core: 5e

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [34]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


In [35]:
launch_dict.keys()


dict_keys(['FlightNumber', 'Date', 'BoosterVersion', 'PayloadMass', 'Orbit', 'LaunchSite', 'Outcome', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial', 'Longitude', 'Latitude'])

Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [36]:
# Create a data from launch_dict
df = pd.DataFrame(launch_dict)

Show the summary of the dataframe


In [37]:
df.shape

(94, 17)

In [38]:
df.describe()

,FlightNumber,PayloadMass,Flights,Block,ReusedCount,Longitude,Latitude
count,94.000000,88.000000,94.000000,90.000000,94.000000,94.000000,94.000000
mean,54.202128,5919.165341,1.755319,3.500000,3.053191,-75.553302,28.581782
std,30.589048,4909.689575,1.197544,1.595288,4.153938,53.391880,4.639981
min,1.000000,20.000000,1.000000,1.000000,0.000000,-120.610829,9.047721
25%,28.250000,2406.250000,1.000000,2.000000,0.000000,-80.603956,28.561857
50%,52.500000,4414.000000,1.000000,4.000000,1.000000,-80.577366,28.561857
75%,81.500000,9543.750000,2.000000,5.000000,4.000000,-80.577366,28.608058
max,106.000000,15600.000000,6.000000,5.000000,13.000000,167.743129,34.632093


In [39]:
# Show the head of the dataframe
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
9,13,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1005,-80.577366,28.561857


### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [40]:
# Hint data['BoosterVersion']!='Falcon 1'
falcon_9_df = df[df["BoosterVersion"] == "Falcon 9"]
falcon_9_df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
9,13,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1005,-80.577366,28.561857
10,14,2014-04-18,Falcon 9,2296.0,ISS,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1006,-80.577366,28.561857
11,15,2014-07-14,Falcon 9,1316.0,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1007,-80.577366,28.561857
12,16,2014-08-05,Falcon 9,4535.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1008,-80.577366,28.561857
13,17,2014-09-07,Falcon 9,4428.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1011,-80.577366,28.561857


Now that we have removed some values we should reset the FlgihtNumber column


In [41]:
falcon_9_df.loc[:,'FlightNumber'] = list(range(1, falcon_9_df.shape[0]+1))
falcon_9_df

/tmp/wsuser/ipykernel_164/467614568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon_9_df.loc[:,'FlightNumber'] = list(range(1, falcon_9_df.shape[0]+1))


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,86,2020-09-03,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058
90,87,2020-10-06,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,13,B1058,-80.603956,28.608058
91,88,2020-10-18,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058
92,89,2020-10-24,Falcon 9,15600.0,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [42]:
print(falcon_9_df.isnull().sum())
falcon_9_df.describe(include="all")

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
count,90.000000,90,90,85.000000,90,90,90,90.000000,90,90,90,64,90.000000,90.000000,90,90.000000,90.000000
unique,NaN,90,1,NaN,11,3,8,NaN,2,2,2,5,NaN,NaN,53,NaN,NaN
top,NaN,2010-06-04,Falcon 9,NaN,GTO,CCSFS SLC 40,True ASDS,NaN,True,False,True,5e9e3032383ecb6bb234e7ca,NaN,NaN,B1049,NaN,NaN
freq,NaN,1,90,NaN,27,55,41,NaN,70,53,71,35,NaN,NaN,6,NaN,NaN
mean,45.500000,NaN,NaN,6123.547647,NaN,NaN,NaN,1.788889,NaN,NaN,NaN,NaN,3.500000,3.188889,NaN,-86.366477,29.449963
std,26.124701,NaN,NaN,4870.916417,NaN,NaN,NaN,1.213172,NaN,NaN,NaN,NaN,1.595288,4.194417,NaN,14.149518,2.141306
min,1.000000,NaN,NaN,350.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,-120.610829,28.561857
25%,23.250000,NaN,NaN,2482.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,2.000000,0.000000,NaN,-80.603956,28.561857
50%,45.500000,NaN,NaN,4535.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,4.000000,1.000000,NaN,-80.577366,28.561857
75%,67.750000,NaN,NaN,9600.000000,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,5.000000,4.000000,NaN,-80.577366,28.608058


Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [57]:
# Calculate the mean value of PayloadMass column
payloadmass_mean = falcon_9_df["PayloadMass"].mean()

falcon_9_df["PayloadMass"] = falcon_9_df["PayloadMass"].replace(np.NaN, payloadmass_mean)
# Replace the np.nan values with its mean value


/tmp/wsuser/ipykernel_164/2166858121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon_9_df["PayloadMass"] = falcon_9_df["PayloadMass"].replace(np.NaN, payloadmass_mean)


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


In [60]:
falcon_9_df.to_csv('dataset_part_1.csv', index=False)


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
